# NNodely Documentation - Local Model

Represents a Local Model relation in the neural network model. The LocalModel structure block is a way to handle non-linear relationships using the fuzzy logic inside the neural network architecture.

In [ ]:
# uncomment the command below to install the nnodely package
#!pip install nnodely

from nnodely import *
from nnodely.relation import NeuObj

## Example 1 - Basic usage

Create a local model with a triangular activation function that represent the one-hot encoding of the discrete variable 'c' and a Fir function for the input 'x'

In [ ]:
c = Input('c')
x = Input('x')
activation = Fuzzify(2,[0,1],functions='Triangular')(c.last())
loc = LocalModel(input_function=Fir())
out = Output('out', loc(x.tw(1), activation))

## Example 2 - passing an output function

We can also define an output function where only the 'active' nodes will be updated.

(Note: if we want a different function (with different weights) for each activation input we have to use the built-in 'lambda' keyword)

In [ ]:
c = Input('c')
x = Input('x')
activation = Fuzzify(2,[0,1],functions='Triangular')(c.last())
loc = LocalModel(input_function = lambda:Fir, output_function = lambda:Fir)(x.last(), activation)
out = Output('out', loc)

## Example 3 - passing a custom function

The LocalModel also accept Parametric functions created using custom user-defined functions.

In [ ]:
def myFun(in1,p1,p2):
    return p1*in1+p2

c = Input('c')
x = Input('x')
activation = Fuzzify(2,[0,1],functions='Triangular')(c.last())
loc = LocalModel(input_function = lambda:ParamFun(myFun), output_function = lambda:Fir)(x.last(), activation)
out = Output('out', loc)

## Example 4 - custom function with multiple activations

We can use multiple activation functions to define the fuzzy logic of the local model. If this is the case, just include all the activations inside a tuple to pass to the local model.

In [ ]:
c = Input('c')
d = Input('d')
activationA = Fuzzify(2,[0,1],functions='Triangular')(c.tw(1))
activationB = Fuzzify(2,[0,1],functions='Triangular')(d.tw(1))

def myFun(in1,p1,p2):
    return p1*in1+p2

x = Input('x')
loc = LocalModel(input_function = lambda:ParamFun(myFun), output_function = Fir(3))(x.tw(1),(activationA,activationB))
out = Output('out', loc)

## Example 5 - Passing indexes

By setting the 'pass_indexes' attribute to True we are indicating whether to pass indexes to the functions

In [ ]:
c = Input('c')
d = Input('d')
activationA = Fuzzify(2,[0,1],functions='Triangular')(c.tw(1))
activationB = Fuzzify(2,[0,1],functions='Triangular')(d.tw(1))

def myFun(in1,p1,p2):
    return p1*in1+p2

def input_function_gen(idx_list):
    if idx_list == [0,0]:
        p1, p2 = Parameter('p1_0',values=[[1]]), Parameter('p2_0',values=[[2]])
    if idx_list == [0,1]:
        p1, p2 = Parameter('p1_0',values=[[1]]), Parameter('p2_1',values=[[3]])
    if idx_list == [1,0]:
        p1, p2 = Parameter('p1_1',values=[[2]]), Parameter('p2_0',values=[[2]])
    if idx_list == [1, 1]:
        p1, p2 = Parameter('p1_1',values=[[2]]), Parameter('p2_1',values=[[3]])
    return ParamFun(myFun,parameters_and_constants=[p1,p2])

def output_function_gen(idx_list):
    pfir = Parameter('pfir_'+str(idx_list),tw=1,dimensions=2,values=[[1+idx_list[0],2+idx_list[1]],[3+idx_list[0],4+idx_list[1]]])
    return Fir(2,W=pfir)

x = Input('x')
loc = LocalModel(input_function=input_function_gen, output_function= output_function_gen, pass_indexes = True)(x.tw(1),(activationA,activationB))
out = Output('out', loc)